# Quantum QSVC Training
## Multiclass Performance with Single Label Training & Prediction

This notebook represents the Quantum Support Vector Classifier side of experiments.

In [ ]:
# For dataset.
from sklearn.datasets import make_multilabel_classification, make_classification
from qiskit_algorithms.utils import algorithm_globals
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC, QSVC
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.kernels import BaseKernel, TrainableFidelityQuantumKernel

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import pandas as pd
import abc

In [ ]:
# for data split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap
from qiskit.circuit.library import RealAmplitudes

In [ ]:
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel, FidelityStatevectorKernel

----

In [ ]:
def conv_to_int(vector):
        val = "".join(vector.astype(str))
        value = int(val, 2)
        return value

In [ ]:
def int_to_hot(x, no_of_classes):
        format = '{' + '0:0{:d}b'.format(no_of_classes) + '}'
        result = format.format(x)
        list_of_ints = [int(x) for x in result]
        return np.array(list_of_ints)

In [ ]:
def get_mapping(no_of_classes):
        dictionary = {}
        numbers = list(range(0, 2**no_of_classes))
        for i in numbers:
            dictionary[i] = int_to_hot(i, no_of_classes)
        return dictionary

In [ ]:
# get data
def data_generation(n_samples, n_features, n_classes, n_labels):
        """
        Generate classification data using sklearn's data generation.\
        """
        X, y = make_multilabel_classification(n_samples=n_samples, 
                                              n_features=n_features, 
                                              n_classes=n_classes, 
                                              n_labels=n_labels,
                                              random_state=algorithm_globals.random_seed
                                             )
        y_new = np.array([conv_to_int(val) for val in y])
        print(X.shape, y.shape, y_new.shape)
        print(X[:5], y[:5], y_new[:5])
        return X, y, y_new

In [ ]:
results = {}
model = 'QSVC Kernel'
no_of_samples = 1024
no_of_features = [2,4,8,12] # as much as quantum side allows
no_of_classes = [3] 
no_of_labels = [3] 
no_of_featuremap_reps = [2,3,4,5,6,8,12]

In [ ]:
for no_feature in no_of_features:
    for reps in no_of_featuremap_reps:
        for no_class in no_of_classes:
            for no_label in no_of_labels:
                if no_label <= no_class:
                    X, y, y_new = data_generation(no_of_samples, no_feature, no_class, no_label)
                    adhoc_feature_map = ZZFeatureMap(feature_dimension=no_feature, reps=reps, entanglement="linear")
                    adhoc_kernel = FidelityStatevectorKernel(feature_map=adhoc_feature_map)
                    optimizer = COBYLA(maxiter=50)
                    qsvc = QSVC(quantum_kernel=adhoc_kernel)
                    start = time.time()
                    qsvc.fit(X, y_new)
                    elapsed = time.time() - start
                    predictions = qsvc.predict(X)
                    mapping = get_mapping(no_class)
                    predictions_labels_final = np.array([mapping[val] for val in predictions])
                    print(predictions_labels_final[:5])
                    accuracy_scores = accuracy_score(y, predictions_labels_final)
                    f1_scores = f1_score(y, predictions_labels_final, average='weighted')
                    results.setdefault('Model', []).append(model)
                    results.setdefault('No of features', []).append(no_feature)
                    results.setdefault('No of samples', []).append(no_of_samples)
                    results.setdefault('No of classes', []).append(no_class)
                    results.setdefault('No of labels', []).append(no_label)
                    results.setdefault('No of feature map reps', []).append(reps)
                    results.setdefault('Accuracy', []).append(accuracy_scores)
                    results.setdefault('F1', []).append(f1_scores)
                    results.setdefault('Time taken', []).append(elapsed)

In [ ]:
df = pd.DataFrame(results)
df.to_csv(f'''results/Final/QSVC-experiments.csv''', index=False)